<a href="https://colab.research.google.com/github/PhamBaVuong/PhamBaVuong_DATN/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scitime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scitime-0.1.1.tar.gz (15 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from __future__ import print_function

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import time

batch_size = 64  # Batch size for training.
epochs = 2000  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/content/drive/MyDrive/duong.txt'


# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
#target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('<astart>')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in input_characters:
            input_characters.add(char)

input_characters = sorted(list(input_characters))

num_encoder_tokens = len(input_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])

max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)

print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
#target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_encoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, input_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, input_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    decoder_target_data[i, t:, input_token_index[' ']] = 1.



# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_encoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_encoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
start = time.time()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, validation_split=0.1)
end = time.time()
print ('Estimate time: ', end - start)
# Save model
model.save('s2s15.h5')


# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
#reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_encoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, input_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_input_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_encoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(0, 20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Input sentence:', target_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)



Number of samples: 140
Number of unique input tokens: 107
Max sequence length for inputs: 20
Max sequence length for outputs: 40
Epoch 1/2000
2/2 [==============================] - 6s 1s/step - loss: 4.6315 - accuracy: 0.0863 - val_loss: 4.0193 - val_accuracy: 0.1804
Epoch 2/2000
2/2 [==============================] - 1s 382ms/step - loss: 3.4496 - accuracy: 0.1423 - val_loss: 2.6908 - val_accuracy: 0.0982
Epoch 3/2000
2/2 [==============================] - 1s 400ms/step - loss: 2.6186 - accuracy: 0.1349 - val_loss: 2.5199 - val_accuracy: 0.1321
Epoch 4/2000
2/2 [==============================] - 1s 373ms/step - loss: 2.5002 - accuracy: 0.1385 - val_loss: 2.4888 - val_accuracy: 0.1304
Epoch 5/2000
2/2 [==============================] - 1s 370ms/step - loss: 2.4544 - accuracy: 0.1526 - val_loss: 2.4346 - val_accuracy: 0.1679
Epoch 6/2000
2/2 [==============================] - 1s 386ms/step - loss: 2.4171 - accuracy: 0.1714 - val_loss: 2.4154 - val_accuracy: 0.1446
Epoch 7/2000
2/2 [====

In [ ]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import numpy as np


class Seq2Seq():
    def __init__(self):
        self.batch_size = 32  # Batch size for training.
        self.epochs = 1000  # Number of epochs to train for.
        self.latent_dim = 256  # Latent dimensionality of the encoding space.
        self.num_samples = 10000  # Number of samples to train on.
        # Path to the data txt file on disk.
        self.data_path = '/content/drive/MyDrive/duong.txt'
        # Vectorize the data.
        self.input_texts = []
        self.target_texts = []
        self.input_characters = set()
        self.num_encoder_tokens = 0
        self.max_encoder_seq_length = 0
        self.max_decoder_seq_length = 0

        self.input_token_index = dict()

        self.encoder_model = Model()
        self.decoder_model = Model()

    def makeVocabulary(self):
        # target_characters = set()
        with open(self.data_path, 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')
        for line in lines[: min(self.num_samples, len(lines) - 1)]:
            input_text, target_text = line.split('<astart>')
            # We use "tab" as the "start sequence" character
            # for the targets, and "\n" as "end sequence" character.
            # input_text = input_text.replace(u'\ufeff', '')
            target_text = '\t' + target_text + '\n'
            self.input_texts.append(input_text)
            self.target_texts.append(target_text)
            for char in input_text:
                if char not in self.input_characters:
                    self.input_characters.add(char)
            for char in target_text:
                if char not in self.input_characters:
                    self.input_characters.add(char)

        self.input_characters = sorted(list(self.input_characters))

    def makeHotEncoding(self):
        self.num_encoder_tokens = len(self.input_characters)
        self.max_encoder_seq_length = max([len(txt) for txt in self.input_texts])
        self.max_decoder_seq_length = max([len(txt) for txt in self.target_texts])
        self.input_token_index = dict(
            [(char, i) for i, char in enumerate(self.input_characters)])
        # target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
        encoder_input_data = np.zeros(
            (len(self.input_texts), self.max_encoder_seq_length, self.num_encoder_tokens),
            dtype='float32')
        decoder_input_data = np.zeros(
            (len(self.input_texts), self.max_decoder_seq_length, self.num_encoder_tokens),
            dtype='float32')
        decoder_target_data = np.zeros(
            (len(self.input_texts), self.max_decoder_seq_length, self.num_encoder_tokens),
            dtype='float32')

        for i, (input_text, target_text) in enumerate(zip(self.input_texts, self.target_texts)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t, self.input_token_index[char]] = 1.
            encoder_input_data[i, t + 1:, self.input_token_index[' ']] = 1.
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, self.input_token_index[char]] = 1.
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, self.input_token_index[char]] = 1.
            decoder_input_data[i, t + 1:, self.input_token_index[' ']] = 1.
            decoder_target_data[i, t:, self.input_token_index[' ']] = 1.

        self.reverse_input_char_index = dict(
            (i, char) for char, i in self.input_token_index.items())
        return encoder_input_data, decoder_input_data, decoder_target_data

    def loadSeq2SeqModel(self):

        model = load_model('/content/drive/MyDrive/s2s15.h5')
        # Define an input sequence and process it.
        encoder_inputs = model.input[0]
        encoder_outputs, state_h, state_c = model.layers[2].output
        # We discard `encoder_outputs` and only keep the states.
        encoder_states = [state_h, state_c]
        self.encoder_model = Model(encoder_inputs, encoder_states)
        # Set up the decoder, using `encoder_states` as initial state.
        decoder_inputs = model.input[1]
        # We set up our decoder to return full output sequences,
        # and to return internal states as well. We don't use the
        # return states in the training model, but we will use them in inference.
        decoder_state_input_h = Input(shape=(self.latent_dim,), name='input_3')
        decoder_state_input_c = Input(shape=(self.latent_dim,), name='input_4')
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

        decoder_lstm = model.layers[3]

        decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs,
                                                                 initial_state=decoder_states_inputs)
        decoder_states = [state_h_dec, state_c_dec]
        decoder_dense = model.layers[4]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        # Define the model that will turn
        # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

        # Next: inference mode (sampling).
        # Here's the drill:
        # 1) encode input and retrieve initial decoder state
        # 2) run one step of decoder with this initial state
        # and a "start of sequence" token as target.
        # Output will be the next target token
        # 3) Repeat with the current target token and current states

        # Define sampling models
        # Reverse-lookup token index to decode sequences back to
        # something readable.

        # reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    def decode_sequence(self, input_seq):
        # Encode the input as state vectors.
        encoder_model = self.encoder_model
        decoder_model = self.decoder_model

        states_value = encoder_model.predict(input_seq)

        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, self.num_encoder_tokens))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0, self.input_token_index['\t']] = 1.

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = ''
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict(
                [target_seq] + states_value)

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = self.reverse_input_char_index[sampled_token_index]
            decoded_sentence += sampled_char

            # Exit condition: either hit max length
            # or find stop character.
            if (sampled_char == '\n' or
                    len(decoded_sentence) > self.max_decoder_seq_length):
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, self.num_encoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.

            # Update states
            states_value = [h, c]

        return decoded_sentence

    def str2HotCoding(self, sentence):
        sequence = np.zeros((1, self.max_encoder_seq_length, self.num_encoder_tokens), dtype='float32')
        for t, char in enumerate(sentence):
            sequence[0, t, self.input_token_index[char]] = 1.
        sequence[0, t + 1:, self.input_token_index[' ']] = 1.
        return sequence


seq = Seq2Seq()
seq.makeVocabulary()
encoder_input_data, _, _ = seq.makeHotEncoding()
s = seq.str2HotCoding('Alexandre de Rhodes')
input = s[0:1]
seq.loadSeq2SeqModel()
out = seq.decode_sequence(input)
print(out)


10.779634866903798, 106.69683645007238



In [ ]:
pip install scitime==0.1.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scitime-0.1.1.tar.gz (15 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import time

from scitime import RuntimeEstimator

# example for kmeans clustering
estimator = RuntimeEstimator(meta_algo='RF', verbose=3)
km = KMeans()

X = np.random.rand(100000,10)
# run the estimation
estimation, lower_bound, upper_bound = estimator.time(km, X)

# compare to the actual training time
start_time = time.time()
km.fit(X)
elapsed_time = time.time() - start_time
print("elapsed time: {:.2}".format(elapsed_time))

ModuleNotFoundError: ignored

In [5]:
from __future__ import print_function

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import time

batch_size = 64  # Batch size for training.
epochs = 150  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '/content/drive/MyDrive/diaDiemQ1_2.txt'


# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
#target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('<astart>')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in input_characters:
            input_characters.add(char)

input_characters = sorted(list(input_characters))

num_encoder_tokens = len(input_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])

max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)

print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
#target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_encoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, input_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, input_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    decoder_target_data[i, t:, input_token_index[' ']] = 1.



# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_encoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_encoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
start = time.time()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, validation_split=0.1)
end = time.time()
print ('Estimate time: ', end - start)
# Save model
model.save('s3s16.h5')


# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
#reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_encoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, input_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_input_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_encoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(0, 20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Input sentence:', target_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)



Number of samples: 140
Number of unique input tokens: 18
Max sequence length for inputs: 38
Max sequence length for outputs: 53
Epoch 1/150
2/2 [==============================] - 7s 1s/step - loss: 2.6374 - accuracy: 0.4702 - val_loss: 0.5005 - val_accuracy: 0.9811
Epoch 2/150
2/2 [==============================] - 1s 486ms/step - loss: 0.6437 - accuracy: 0.9245 - val_loss: 0.0884 - val_accuracy: 0.9811
Epoch 3/150
2/2 [==============================] - 1s 482ms/step - loss: 0.4382 - accuracy: 0.9245 - val_loss: 0.1001 - val_accuracy: 0.9811
Epoch 4/150
2/2 [==============================] - 1s 486ms/step - loss: 0.4030 - accuracy: 0.9245 - val_loss: 0.0928 - val_accuracy: 0.9811
Epoch 5/150
2/2 [==============================] - 1s 477ms/step - loss: 0.3946 - accuracy: 0.9245 - val_loss: 0.1486 - val_accuracy: 0.9811
Epoch 6/150
2/2 [==============================] - 1s 486ms/step - loss: 0.4242 - accuracy: 0.9245 - val_loss: 0.1510 - val_accuracy: 0.9636
Epoch 7/150
2/2 [============